这是一个使用CodeFuse-Query分析Go项目的教程。在教程中，你将体验到使用命令行工具对代码仓库进行数据化，然后使用Godel语言来分析这个仓库。

环境准备：配置cli的路径，你可以将sparrow cli包导出到系统PATH，如 `!export PATH=/Users/cyz/work/opensource/cli:$PATH`：

检查cli是否就绪

In [2]:
!which sparrow

/sparrow-cli/sparrow



STEP 0: 克隆要分析的仓库。我们以 [gorm](https://github.com/go-gorm/gorm.git) 项目为例。

In [3]:
!git clone https://github.com/go-gorm/gorm.git

Cloning into 'gorm'...




remote: Enumerating objects: 13402, done.        


remote: Counting objects:   0% (1/4415)        

remote: Counting objects:   1% (45/4415)        

remote: Counting objects:   2% (89/4415)        

remote: Counting objects:   3% (133/4415)        

remote: Counting objects:   4% (177/4415)        

remote: Counting objects:   5% (221/4415)        

remote: Counting objects:   6% (265/4415)        

remote: Counting objects:   7% (310/4415)        

remote: Counting objects:   8% (354/4415)        

remote: Counting objects:   9% (398/4415)        

remote: Counting objects:  10% (442/4415)        

remote: Counting objects:  11% (486/4415)        

remote: Counting objects:  12% (530/4415)        

remote: Counting objects:  13% (574/4415)        

remote: Counting objects:  14% (619/4415)        

remote: Counting objects:  15% (663/4415)        

remote: Counting objects:  16% (707/4415)        

remote: Counting objects:  17% (751/4415)        

remote: Counting objects:  18% (

STEP 1: 代码数据化。使用 `sparrow database create` 命令创建一个db文件，指定待分析的仓库地址（当前目录下的gorm子目录），分析的语言（go），以及db文件的存储路径（放置在当前目录下的/db/gorm）。执行该命令之后，竟会生成一份db文件，该文件存储着代码仓库的结构化数据，之后的分析就是针对这份数据进行。

In [4]:
!sparrow database create --source-root gorm --data-language-type go --output ./db/gorm

2023-11-29 02:16:35,686 INFO: sparrow 2.0.0


 will start


2023-11-29 02:16:35,687 WARNING: /workspaces/codespaces-jupyter/notebooks/db/gorm not exists, it will be created


2023-11-29 02:16:35,687 INFO: /workspaces/codespaces-jupyter/notebooks/db/gorm success build


2023-11-29 02:16:35,687 INFO: current memory is : 15.61 GB


2023-11-29 02:16:35,687 INFO: final -Xmx is : 14.61 GB


2023-11-29 02:16:35,688 INFO: execute : /sparrow-cli/language/go/extractor/coref-go-src-extractor -o /workspaces/codespaces-jupyter/notebooks/db/gorm/coref_go_src.db /workspaces/codespaces-jupyter/notebooks/gorm


2023/11/29 02:16:35 Max goroutines set to 64


2023/11/29 02:16:35 ExtractorRunningParams: &{extractor:/sparrow-cli/language/go/extractor/coref-go-src-extractor extractArgs:[] buildArgs:[] dbPath:/workspaces/codespaces-jupyter/notebooks/db/gorm/ dbName:coref_go_src.db srcRoot:/workspaces/codespaces-jupyter/notebooks/gorm shouldInstallDependency:false}


2023/11/29 02:16:35 extracting go sdk


20

STEP 2: 使用Godel分析语言分析db文件。在本教程中，可以点击代码左侧的执行按钮，直接运行分析脚本。在命令行中，你可以使用 `sparrow query run` 命令来执行查询脚本，具体可以使用`sparrow query run -h` 来获取详细参数信息。

<b>示例</b> 查询 [gorm](https://github.com/go-gorm/gorm.git) 的文件代码复杂度信息。

第一行通过内核魔法命令指定分析的db路径，后面写查询文件代码复杂度Godel脚本。

In [5]:
%db /workspaces/codespaces-jupyter/notebooks/db/gorm
// script
use coref::go::*

fn default_db() -> GoDB {
    return GoDB::load("coref_go_src.db")
}

/**
 * @param name: 文件名
 * @param func: 函数名
 * @param cmplx: 函数圈复杂度
 * @param sl,el,sc,ec: 函数位置信息，依次为函数起始行，结束行
 */
fn out(name: string, func: string, cmplx: int, sl: int, el: int) -> bool {
    for(f in GoFile(default_db()), function in Function(default_db())) {
        if ((!f.isAutoGenereatedFile()) &&
            f.key_eq(function.getBelongsFile()) &&
            name = f.getName() &&
            func = function.getName() &&
            cmplx = function.getCyclomaticComplexity() &&
            sl = function.getLocation().getStartLineNumber() &&
            el = function.getLocation().getEndLineNumber()) {
            return true
        }
    }
}

fn main() {
    output(out())
}

/workspaces/codespaces-jupyter/notebooks/db/gorm


Sparrow database is set to: /workspaces/codespaces-jupyter/notebooks/db/gorm


2023-11-29 02:17:10,972 INFO: sparrow 2.0.0
 will start
2023-11-29 02:17:10,972 INFO: database /workspaces/codespaces-jupyter/notebooks/db/gorm/coref_go_src.db size: 42.16 MB
2023-11-29 02:17:10,973 INFO: execute : /sparrow-cli/godel-script/usr/bin/godel /tmp/godel-jupyter-4xensdwe/query.gdl -p /sparrow-cli/lib-1.0 -o /tmp/tmpyhwrenxv.gdl --verbose
[-][0.280163 ms] package scan
packages:
coref::cfamily    -> /sparrow-cli/lib-1.0/coref.cfamily.gdl
coref::go         -> /sparrow-cli/lib-1.0/coref.go.gdl
coref::java       -> /sparrow-cli/lib-1.0/coref.java.gdl
coref::javascript -> /sparrow-cli/lib-1.0/coref.javascript.gdl
coref::properties -> /sparrow-cli/lib-1.0/coref.properties.gdl
coref::python     -> /sparrow-cli/lib-1.0/coref.python.gdl
coref::sql        -> /sparrow-cli/lib-1.0/coref.sql.gdl
coref::xml        -> /sparrow-cli/lib-1.0/coref.xml.gdl
modules
+--coref -> coref
|--cfamily -> coref::cfamily
|--go -> coref::go
|--properties -> coref::properties
|--java -> coref::java
|--xml -

,name,func,cmplx,sl,el
0,association.go,Delete,20,197,361
1,association.go,Find,2,53,58
2,association.go,Unscoped,1,44,51
3,association.go,Association,4,21,42
4,association.go,Count,2,367,372
...,...,...,...,...,...
936,tests/associations_has_many_test.go,TestHasManyAssociationForSlice,4,217,269
937,tests/associations_has_many_test.go,TestSingleTableHasManyAssociationForSlice,5,271,325
938,tests/associations_has_many_test.go,TestPolymorphicHasManyAssociation,12,327,420
939,tests/associations_has_many_test.go,TestPolymorphicHasManyAssociationForSlice,4,422,474


保存上一次运行的 query 结果保存到一个JSON文件

In [6]:
%%save_to /workspaces/codespaces-jupyter/notebooks/db/gorm/query.json

Query result saved to /workspaces/codespaces-jupyter/notebooks/db/gorm/query.json


STEP 3: 好了，你可以针对分析生成的结果，进行进一步的代码分析了，比如你可以结合pandas库，使用刚刚生成的 query.json 实现最大函数复杂度的排序查询：

In [7]:
%%python
import pandas as pd
data = pd.read_json('/workspaces/codespaces-jupyter/notebooks/db/gorm/query.json')
data.sort_values('cmplx', ascending=False, inplace=True)
top_10 = data.head(10)
print(top_10)

                      name                       func  cmplx   sl   el
336        schema/field.go                 ParseField    110   99  436
164    callbacks/update.go       ConvertToAssignments     69  131  304
383       schema/schema.go  ParseWithSpecialTableName     68  121  370
121    callbacks/create.go      ConvertToCreateValues     64  217  385
241                scan.go                       Scan     62  125  342
46            statement.go             BuildCondition     61  284  465
337        schema/field.go       setupValuerAndSetter     46  439  969
753  tests/migrate_test.go         TestMigrateColumns     44  517  645
359   migrator/migrator.go              MigrateColumn     41  441  542
168     callbacks/query.go              BuildQuerySQL     41   33  266


Enjoy！